In [1]:
import os, re, json
import torch, numpy as np

import sys
torch.set_grad_enabled(False)

In [2]:
from src.utils.model_utils import load_gpt_model_and_tokeniser

from src.utils.prompt_utils import load_dataset

from src.utils.extract_utils import create_steering_vector

/root/activation-additions-large-models/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'EleutherAI/gpt-j-6b'
model, tokenizer, model_config = load_gpt_model_and_tokeniser(model_name)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
dataset = load_dataset('antonym',
                       'dataset_files',
                       test_size=0.3,
                        seed=0)

In [5]:
stories = {}

# Open and read the JSON file
with open('datasets/fantasy.json', 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

with open('datasets/scifi.json', 'r') as file:
  # Load the JSON data from the file
  dataset_scifi = json.load(file)

  stories["scifi"] = dataset_scifi


from src.utils.dataset_utils import read_all_text_files

training_dataset = read_all_text_files("datasets/opentext_subset")

# Cut texts for first 200 tokens
# Determine the cutoff point using the tokenizer
training_dataset = [tokenizer.decode(tokenizer.encode(text)[:200])[4:] for text in training_dataset][:400]

Token indices sequence length is longer than the specified maximum sequence length for this model (2302 > 2048). Running this sequence through the model will result in indexing errors


In [7]:
steering_vector = create_steering_vector(
    model,
    tokenizer,
    model_config,
    dataset_fantasy,
    training_dataset,
    ["layer_hook_names"],
    False,
    False,
    False
)

Gathering activations:   0%|          | 0/200 [00:00<?, ?it/s]

Gathering activations: 100%|██████████| 400/400 [01:25<00:00,  4.67it/s]


In [9]:
model_config["n_layers"]

28

In [36]:
steering_vector.shape

torch.Size([28, 4096])

In [ ]:
steer

In [46]:
from src.utils.intervention_utils import steering_natural_text

outputs = steering_natural_text(
    "Here is a story:", 
    23,
    steering_vector[23], 
    model,
    model_config, 
    tokenizer, 
    max_new_tokens=100, 
    temperature=1.0, 
    freq_penalty=2.0,
    top_p=0.3,
    n_completions=5,
    n_beams=50
)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [01:35<00:00, 19.16s/it]


In [48]:
outputs['steered'][0]

'\n\n� "We are going to get this guy, and we are going to get the guys on top of that. We are going to get them all. � (I) really think it is going to work. I really do. I really think it is going to work. I really do. I really think it is going to work. I really do. I really think it is going to work. I really do. I really think it is going to work. I really do'

In [ ]:
import gc
gc.collect()
torch.cuda.f